In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [3]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=ColourContext  
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='a0aa9b07-f8fb-4fa6-9098-89ed30bb677f'),
              AIMessage(content='I’m not sure what your favorite colour is—could you let me know?', additional_kwargs={'reasoning_content': 'The user asks "What is my favourite colour?" There\'s no prior context. We don\'t know. According to policy, we must respond that we don\'t know. We can ask clarifying question or say we don\'t know. The user might be expecting a guess, but we shouldn\'t guess. We can ask them to tell us. So answer: I don\'t know, could you tell me?'}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 77, 'total_tokens': 179, 'completion_time': 0.211125059, 'completion_tokens_details': {'reasoning_tokens': 77}, 'prompt_time': 0.002959356, 'prompt_tokens_details': None, 'queue_time': 0.007826445, 'total_time': 0.214084415}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp

## Accessing Context

In [7]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

@tool
def get_favourite_colour(config: RunnableConfig) -> str:
    """Get the favourite colour of the user"""
    # Accessing the context passed via the 'configurable' field
    context = config.get("configurable", {}).get("context")
    if context:
        return context.favourite_colour
    return "I don't know the favourite colour."

@tool
def get_least_favourite_colour(config: RunnableConfig) -> str:
    """Get the least favourite colour of the user"""
    context = config.get("configurable", {}).get("context")
    if context:
        return context.least_favourite_colour
    return "I don't know the least favourite colour."

In [8]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [9]:
config = {
    "configurable": {
        "thread_id": "session_123",
        "context": ColourContext(
            favourite_colour="blue", 
            least_favourite_colour="yellow"
        )
    }
}

In [10]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    config=config
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='a700dcf8-8ad7-4a13-97f4-d8773cff3dcf'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to get favourite colour via function. Use get_favourite_colour.', 'tool_calls': [{'id': 'fc_db1d00b8-fae5-4bd7-83c8-ea657df570b7', 'function': {'arguments': '{}', 'name': 'get_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 149, 'total_tokens': 186, 'completion_time': 0.076390894, 'completion_tokens_details': {'reasoning_tokens': 16}, 'prompt_time': 0.005944842, 'prompt_tokens_details': None, 'queue_time': 0.005039352, 'total_time': 0.082335736}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_309cec8e88', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c44bc-4288-71c2-8bc7-f56e4d4dc069-0', too

In [11]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my least favourite colour?")]},
    config=config
)

pprint(response)

{'messages': [HumanMessage(content='What is my least favourite colour?', additional_kwargs={}, response_metadata={}, id='16a978c4-78bf-415d-8032-5bc8fd1c1535'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to answer user\'s question: "What is my least favourite colour?" We have functions to get favourite and least favourite colour. We should call get_least_favourite_colour.', 'tool_calls': [{'id': 'fc_49a108d4-650d-4dab-be0f-c38f4db736c6', 'function': {'arguments': '{}', 'name': 'get_least_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 150, 'total_tokens': 210, 'completion_time': 0.124541396, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.005621192, 'prompt_tokens_details': None, 'queue_time': 0.004813537, 'total_time': 0.130162588}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_dddd02f37b', 'service_tier': 'on_demand', 'finish_reaso